In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from services.zayev.environment.market_simulator import MarketSimulator
from apps.environment_simulator.models import SimulatedStockBuffer
import gymnasium as gym
import ray
from gymnasium import spaces
from ray.rllib.algorithms import ppo
from os import path, environ
from gymnasium.envs.registration import register
from ray import tune

2023-10-16 09:46:09.031911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 09:46:14,797	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}

In [3]:


ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})

the_current_time_step  = datetime(year=1995, month=1, day=1, hour=10)

config = {
        "env": MarketSimulator,
        "env_config": {
            "db_params": db_params, 
            "max_episode_steps": 150, 
            "the_current_time_step": the_current_time_step,
            "print_output": False,
            "test_steps": 10,
            "n_step_stocks": 5,
            "n_step_cmmdties": 5,
            "epsilon": 0.4
        },
        "num_workers": 4,
        "entropy_coeff": 0.5,
        # "disable_env_checking": True
}

algo = ppo.PPO(config=config)

2023-10-16 09:46:19,727	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-16 09:46:20,442	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-16 09:46:20,944	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_16a0052c6451ea8a.zip' (45.42MiB) to Ray cluster...
2023-10-16 09:46:21,148	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_16a0052c6451ea8a.zip'.
2023-10-16 09:46:23,762	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-pac

In [5]:
zayev = MarketSimulator(env_config={
    "db_params": db_params, 
    "max_episode_steps": 150, 
    "the_current_time_step": the_current_time_step,
    "print_output": False,
    "test_steps": 10,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "epsilon": 0.4
    }
)
zayev.reset()
actn = algo.compute_single_action(zayev.state)
print(actn)

[ 1000.  1000.  1000. -1000.  1000. -1000.]


In [7]:


mean_ppo = []
for _ in range(10):

    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])


episode reward mean: 0 -38.4
episode reward mean: 1 -25.8
episode reward mean: 2 -45.4
episode reward mean: 3 -86.6
episode reward mean: 4 -76.8
episode reward mean: 5 -69.6
episode reward mean: 6 -52.8
episode reward mean: 7 -34.4
episode reward mean: 8 -51.2
episode reward mean: 9 -33.2
episode reward mean: 10 -24.2
episode reward mean: 11 -40.8
episode reward mean: 12 -42.8
episode reward mean: 13 -79.2
episode reward mean: 14 -64.4


OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.
OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 15 -63.6
episode reward mean: 16 -52.4


OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 17 -62.4
episode reward mean: 18 -48.0


OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 19 -56.6


In [5]:
checkpoint_path = "/Users/mirbilal/Desktop/MobCommission/commissionV2/services/zayev/simulated_checkpoints/"

algo.save(checkpoint_dir=checkpoint_path)

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/Users/mirbilal/Desktop/MobCommission/commissionV2/services/zayev/simulated_checkpoints/), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': -61.043173637471476}, 'default_policy': {'total_loss': -61.043173637471476, 'policy_loss': 0.09904743755645336, 'vf_loss': 10.0, 'vf_loss_unclipped': 1770186621.3219616, 'vf_explained_var': -5.347578764470147e-06, 'entropy': 142.70959991585218, 'mean_kl_loss': 0.20995423378691896, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.5, 'curr_kl_coeff': 1.5187500715255737}}, 'num_env_steps_sampled': 20000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 20000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': 308160.5007888747, 'episode_reward_min': -47357.23427858389, 'episode_reward_mean': 42985.256880082954,

In [6]:
algo.restore(checkpoint_path=checkpoint_path)

2023-10-09 10:46:52,822	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


2023-10-09 10:46:52,958	INFO trainable.py:984 -- Restored on 127.0.0.1 from checkpoint: Checkpoint(filesystem=local, path=/Users/mirbilal/Desktop/MobCommission/commissionV2/services/zayev/simulated_checkpoints/)


True

In [4]:
rewards_a = []
rewards_b = []

In [27]:
reward_c = []

In [3]:
my_current_time_step  = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=0)

env_config = {
    "db_params": db_params, 
    "max_episode_steps": 150, 
    "print_output": False,
    "the_current_time_step": my_current_time_step,
}

myEnv = MarketSimulator(env_config)

(1,)
(5, 8)


In [ ]:
ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})

config = {
    "env": MarketSimulator,
    "env_config": {
        "db_params": db_params, 
        "max_episode_steps": 150, 
        "the_current_time_step": my_current_time_step,
        "print_output": False
    },
    "num_workers": 4,
}

new_algo = ppo.PPO(config=config)
new_algo.restore(checkpoint_path=checkpoint_path)

In [41]:

import copy

def get_latest_states():
    latest_states = []
    env = myEnv
    env.reset()
    obs = env.state
    latest_states = []  # To store the latest states from each worker
    for _ in range(150):  # Interact with the environment for 10 steps as an example
        action = algo.compute_single_action(obs)  # Replace with your agent's action
        # action = np.array([[664, 831,  51, 988, 280,],
        #             [  1,   0,   1,   1,   0]])
        obs, _, done, _ = env.step(action)
        if done:
            obs = env.reset()
        state = env.state  # Obtain the state from the environment
        latest_states.append(copy.deepcopy(obs))
    return latest_states

latest_states = get_latest_states()

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


In [42]:
total_portfolios = []
total_rewards = 0
for astate in latest_states:
    # print(astate)
    total_portfolio_a = 0
    total_portfolio_b = 0
    (stock_state, commodity_state, wallet_state) = astate
    total_portfolio_b = total_portfolio_b + wallet_state[0]
    total_portfolio_a = total_portfolio_a + wallet_state[0]
    # print(stock_state)
    gain = 0
    for a_stck in stock_state:
        total_portfolio_a = total_portfolio_a + a_stck[6]*a_stck[7]
        total_portfolio_b = total_portfolio_b + a_stck[0]*a_stck[7]
        gain = gain + (a_stck[0]*a_stck[7] - a_stck[6]*a_stck[7])
    # print(gain)
    # print("")
    total_rewards = total_rewards + gain
    reward_c.append(total_rewards)
        
    # print((total_portfolio_a, total_portfolio_b))
    total_portfolios.append((total_portfolio_a, total_portfolio_b))

print("total_rewards", total_rewards)

total_rewards 4368.091816217878


In [14]:
dividend = 21
divisor = 2

factor = dividend // divisor  # Integer division
remainder = dividend % divisor  # Modulo operation

print(f"Factor (Quotient): {factor}")
print(f"Remainder: {remainder}")

Factor (Quotient): 10
Remainder: 1


In [55]:
ray.shutdown()

In [43]:
cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

buffer = SimulatedStockBuffer.objects.select_related("stock").order_by("captured_at").all()

price_variations = {"reward_c": reward_c}

# # print(price_variations)
# for stck_name in price_variations:
#     price_snaps = price_variations[stck_name]
#     price_array = np.array(price_snaps)
#     max_price = max(price_array)
#     normalized_snaps = price_array/max_price
#     price_variations[stck_name] = normalized_snaps

    

stock_names = ["reward_c"]


# print(price_variations)
df_regret_comparisons = pd.DataFrame(price_variations)
df_regret_comparisons.iplot(
    xTitle='time_steps',
    yTitle='Normalized Prices'
)